In [10]:
# ====================================================================
# ETL COMPLETO PARA NUEVO UNIVERSO DE CRYPTOS CON FUNDING RATES
# ====================================================================
import pandas as pd
import numpy as np
import yfinance as yf
import sqlalchemy
# import pandas_ta as ta  # Comentado por conflicto con numpy
import os
import requests
from datetime import datetime, timedelta
import time

print("--- [INICIO] Construyendo DataFrame para el Nuevo Universo ---")

# --- CONFIGURACIÓN ---
new_ticker_list = [
    'AVAX-USD', 'ATOM-USD', 'NEAR-USD', 'FTM-USD', 'UNI-USD',
    'AAVE-USD', 'ALGO-USD', 'TRX-USD', 'XLM-USD', 'XTZ-USD'
]
start_date = "2020-12-01"
end_date = "2022-04-17"

# Mapeo de tickers de Yahoo Finance a símbolos de Binance para funding rates
TICKER_TO_BINANCE = {
    'AVAX-USD': 'AVAXUSDT',
    'ATOM-USD': 'ATOMUSDT', 
    'NEAR-USD': 'NEARUSDT',
    'FTM-USD': 'FTMUSDT',
    'UNI-USD': 'UNIUSDT',
    'AAVE-USD': 'AAVEUSDT',
    'ALGO-USD': 'ALGOUSDT',
    'TRX-USD': 'TRXUSDT',
    'XLM-USD': 'XLMUSDT',
    'XTZ-USD': 'XTZUSDT'
}

# --- PASO 1: Descargar Precios ---
print("\n -> Paso 1: Descargando datos de precios...")
try:
    df_raw = yf.download(new_ticker_list, start=start_date, end=end_date, progress=False)
    
    if df_raw.empty:
        raise Exception("No se descargaron datos de precios")
    
    df_processed = df_raw.stack(level=1).reset_index()
    df_processed = df_processed.rename(columns={
        'Date': 'timestamp', 'Ticker': 'ticker', 'Open': 'open',
        'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'
    })
    
    master_df = pd.DataFrame({
        'timestamp': pd.to_datetime(df_processed['timestamp'], utc=True),
        'ticker': df_processed['ticker'],
        'open': df_processed['open'],
        'high': df_processed['high'],
        'low': df_processed['low'],
        'close': df_processed['close'],
        'volume': pd.to_numeric(df_processed['volume'], errors='coerce').fillna(0)
    })
    
    master_df.dropna(subset=['close', 'timestamp', 'ticker'], inplace=True)
    print(f"✅ Precios descargados ({master_df.shape[0]} filas).")
    
except Exception as e:
    print(f"❌ Error descargando precios: {e}")
    exit(1)

# --- PASO 2: Función para obtener Funding Rates de Binance ---
def get_funding_rate_history(symbol, start_timestamp, end_timestamp):
    """
    Obtiene el historial de funding rates de Binance
    """
    url = "https://fapi.binance.com/fapi/v1/fundingRate"
    
    funding_rates = []
    current_start = start_timestamp
    
    while current_start < end_timestamp:
        params = {
            'symbol': symbol,
            'startTime': current_start,
            'endTime': min(current_start + (1000 * 8 * 60 * 60 * 1000), end_timestamp),  # Max 1000 records
            'limit': 1000
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if not data:
                break
                
            funding_rates.extend(data)
            current_start = data[-1]['fundingTime'] + 1
            time.sleep(0.1)  # Para evitar rate limiting
            
        except Exception as e:
            print(f"⚠️ Error obteniendo funding rates para {symbol}: {e}")
            break
    
    return funding_rates

print("\n -> Paso 2: Descargando funding rates de Binance...")

# Convertir fechas a timestamps
start_timestamp = int(pd.to_datetime(start_date).timestamp() * 1000)
end_timestamp = int(pd.to_datetime(end_date).timestamp() * 1000)

# Diccionario para almacenar funding rates por ticker
funding_data = {}

for yahoo_ticker, binance_symbol in TICKER_TO_BINANCE.items():
    print(f"  -> Descargando funding rates para {yahoo_ticker} ({binance_symbol})...")
    
    try:
        rates = get_funding_rate_history(binance_symbol, start_timestamp, end_timestamp)
        
        if rates:
            df_funding = pd.DataFrame(rates)
            df_funding['timestamp'] = pd.to_datetime(df_funding['fundingTime'], unit='ms', utc=True)
            df_funding['funding_rate'] = pd.to_numeric(df_funding['fundingRate'])
            df_funding['ticker'] = yahoo_ticker
            
            # Mantener solo las columnas necesarias
            df_funding = df_funding[['timestamp', 'ticker', 'funding_rate']]
            funding_data[yahoo_ticker] = df_funding
            print(f"    ✅ {len(df_funding)} registros de funding rate obtenidos")
        else:
            print(f"    ⚠️ No se encontraron funding rates para {binance_symbol}")
            # Crear DataFrame vacío con funding_rate = 0
            dates = pd.date_range(start=start_date, end=end_date, freq='8H', tz='UTC')
            df_funding = pd.DataFrame({
                'timestamp': dates,
                'ticker': yahoo_ticker,
                'funding_rate': 0.0
            })
            funding_data[yahoo_ticker] = df_funding
            
    except Exception as e:
        print(f"    ❌ Error con {yahoo_ticker}: {e}")
        # Crear DataFrame de respaldo con funding_rate = 0
        dates = pd.date_range(start=start_date, end=end_date, freq='8H', tz='UTC')
        df_funding = pd.DataFrame({
            'timestamp': dates,
            'ticker': yahoo_ticker,
            'funding_rate': 0.0
        })
        funding_data[yahoo_ticker] = df_funding

# Combinar todos los funding rates
if funding_data:
    all_funding_df = pd.concat(funding_data.values(), ignore_index=True)
    print(f"✅ Funding rates procesados ({len(all_funding_df)} registros totales)")
else:
    print("⚠️ No se pudieron obtener funding rates, usando valores por defecto")
    all_funding_df = pd.DataFrame()

# --- PASO 3: Calcular Indicadores Técnicos ---
print("\n -> Paso 3: Calculando indicadores técnicos...")
master_df.sort_values(by=['ticker', 'timestamp'], inplace=True)

def calculate_ema(prices, period):
    """Calcula EMA manualmente"""
    alpha = 2 / (period + 1)
    ema = np.zeros_like(prices)
    ema[0] = prices[0]
    
    for i in range(1, len(prices)):
        ema[i] = alpha * prices[i] + (1 - alpha) * ema[i-1]
    
    return ema

def calculate_macd(prices, fast=12, slow=26, signal=9):
    """Calcula MACD manualmente"""
    ema_fast = calculate_ema(prices, fast)
    ema_slow = calculate_ema(prices, slow)
    macd_line = ema_fast - ema_slow
    macd_signal = calculate_ema(macd_line, signal)
    macd_hist = macd_line - macd_signal
    
    return macd_line, macd_signal, macd_hist

def calculate_indicators(group):
    """Calcula los indicadores técnicos necesarios"""
    try:
        prices = group['close'].values
        
        # Calcular MACD
        macd_line, macd_signal, macd_hist = calculate_macd(prices)
        group['macd'] = macd_line
        group['macd_signal'] = macd_signal
        group['macd_hist'] = macd_hist
        
        # Calcular EMA 26
        group['ema_26'] = calculate_ema(prices, 26)
        
        return group
    except Exception as e:
        print(f"⚠️ Error calculando indicadores para {group['ticker'].iloc[0]}: {e}")
        return group

# Aplicar cálculo de indicadores (sin renombrar ya que usamos nombres correctos)
master_df = master_df.groupby('ticker', group_keys=False).apply(calculate_indicators)

print("✅ Indicadores técnicos calculados")

# --- PASO 4: Cargar Datos Macro desde BBDD ---
print("\n -> Paso 4: Cargando datos macro desde la base de datos...")
try:
    # Configuración de la base de datos
    DB_USER = "cryptonita_user"
    DB_PASSWORD = "TIZavoltio999" 
    DB_HOST = "localhost"
    DB_PORT = "5432"
    DB_NAME = "cryptonita_db2"
    
    db_url = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = sqlalchemy.create_engine(db_url)
    
    # Queries para datos macro
    macro_queries = {
        'macro_spy': "SELECT timestamp, close AS spy_close FROM macro_spy",
        'macro_vix': "SELECT timestamp, close AS vix_close FROM macro_vix", 
        'macro_tnx': "SELECT timestamp, close AS tnx_close FROM macro_tnx",
        'macro_dxy': "SELECT timestamp, close AS dxy_close FROM macro_dx_y_nyb",
        'macro_gc': "SELECT timestamp, close AS gc_close FROM macro_gc",
        'macro_cl': "SELECT timestamp, close AS cl_close FROM macro_cl"
    }
    
    # Cargar datos macro
    macro_dfs = {}
    for name, query in macro_queries.items():
        try:
            df = pd.read_sql(query, engine)
            df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).dt.normalize()
            macro_dfs[name] = df
            print(f"  ✅ {name}: {len(df)} registros cargados")
        except Exception as e:
            print(f"  ⚠️ Error cargando {name}: {e}")
    
    # Unir datos macro al DataFrame principal
    for name, df in macro_dfs.items():
        master_df = pd.merge(master_df, df, on='timestamp', how='left')
    
    print("✅ Datos macro unidos al DataFrame principal")
    
except Exception as e:
    print(f"⚠️ Error con la base de datos: {e}")
    print("  -> Continuando sin datos macro (se llenarán con NaN)")

# --- PASO 5: Unir Funding Rates ---
print("\n -> Paso 5: Uniendo funding rates...")
if not all_funding_df.empty:
    # Normalizar timestamp de funding rates para hacer join diario
    all_funding_df['date'] = all_funding_df['timestamp'].dt.normalize()
    master_df['date'] = master_df['timestamp'].dt.normalize()
    
    # Promediar funding rates por día si hay múltiples registros
    daily_funding = all_funding_df.groupby(['ticker', 'date'])['funding_rate'].mean().reset_index()
    
    # Unir con el DataFrame principal
    master_df = pd.merge(master_df, daily_funding, on=['ticker', 'date'], how='left')
    master_df.drop('date', axis=1, inplace=True)
    
    # Rellenar valores faltantes de funding_rate
    master_df['funding_rate'] = master_df.groupby('ticker')['funding_rate'].fillna(method='ffill').fillna(0)
    
    print("✅ Funding rates unidos")
else:
    master_df['funding_rate'] = 0.0
    print("⚠️ Usando funding_rate = 0 por defecto")

# --- PASO 6: Crear Features Adicionales ---
print("\n -> Paso 6: Creando features adicionales...")
master_df.sort_values(by=['ticker', 'timestamp'], inplace=True)

def create_additional_features(group):
    """Crea las features adicionales necesarias"""
    try:
        # Log return
        group['log_return'] = np.log(group['close'] / group['close'].shift(1))
        
        # Volatilidad 7 días
        group['volatility_7d'] = group['log_return'].rolling(window=7, min_periods=1).std()
        
        # Price to EMA ratio
        if 'ema_26' in group.columns:
            group['price_to_ema_ratio'] = (group['close'] / group['ema_26']) - 1
        else:
            group['price_to_ema_ratio'] = 0
        
        # MACD normalizado
        if 'macd' in group.columns:
            group['macd_norm'] = group['macd'] / group['close']
        else:
            group['macd_norm'] = 0
            
        # Log return del oro (si existe)
        if 'gc_close' in group.columns:
            group['log_return_gc_close'] = np.log(group['gc_close'] / group['gc_close'].shift(1))
        else:
            group['log_return_gc_close'] = 0
            
        return group
    except Exception as e:
        print(f"⚠️ Error creando features para {group['ticker'].iloc[0]}: {e}")
        return group

master_df = master_df.groupby('ticker', group_keys=False).apply(create_additional_features)

# Forward-fill datos macro
macro_cols = ['spy_close', 'vix_close', 'tnx_close', 'dxy_close', 'gc_close', 'cl_close']
for col in macro_cols:
    if col in master_df.columns:
        master_df[col] = master_df[col].fillna(method='ffill')

print("✅ Features adicionales creadas")

# --- PASO 7: Aplicar Look-ahead Bias Correction ---
print("\n -> Paso 7: Aplicando corrección de look-ahead bias...")
feature_cols = ['macd_signal', 'macd_hist', 'funding_rate', 'spy_close', 'vix_close', 
                'tnx_close', 'dxy_close', 'gc_close', 'cl_close', 'log_return', 
                'volatility_7d', 'price_to_ema_ratio', 'macd_norm', 'log_return_gc_close']

existing_feature_cols = [col for col in feature_cols if col in master_df.columns]
master_df[existing_feature_cols] = master_df.groupby('ticker')[existing_feature_cols].shift(1)

# --- PASO 8: Filtrar Solo las Columnas Necesarias ---
target_columns = ['close', 'macd_signal', 'macd_hist', 'funding_rate', 'spy_close', 
                 'vix_close', 'tnx_close', 'dxy_close', 'gc_close', 'cl_close', 
                 'log_return', 'volatility_7d', 'price_to_ema_ratio', 'macd_norm', 
                 'log_return_gc_close']

# Mantener también timestamp y ticker para referencia
final_columns = ['timestamp', 'ticker'] + target_columns
available_columns = [col for col in final_columns if col in master_df.columns]

final_df = master_df[available_columns].copy()

# Eliminar filas con NaN
final_df.dropna(inplace=True)
final_df.reset_index(drop=True, inplace=True)

print(f"✅ Look-ahead bias corregido y columnas filtradas")

# --- PASO 9: Guardar Resultado ---
print("\n -> Paso 9: Guardando DataFrame final...")
output_dir = 'dataframes/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

file_path = os.path.join(output_dir, 'master_df_new_universe_complete.parquet')
final_df.to_parquet(file_path)

print(f"\n🎉 ¡ÉXITO COMPLETO!")
print(f"📁 DataFrame guardado en: {file_path}")
print(f"📊 Shape final: {final_df.shape}")
print(f"📋 Columnas finales: {list(final_df.columns)}")

# Mostrar resumen por ticker
print(f"\n--- RESUMEN POR TICKER ---")
summary = final_df.groupby('ticker').agg({
    'close': 'count',
    'timestamp': ['min', 'max']
}).round(2)
summary.columns = ['Records', 'Start_Date', 'End_Date']
print(summary)

print(f"\n--- MUESTRA DE DATOS ---")
print(final_df.head())

--- [INICIO] Construyendo DataFrame para el Nuevo Universo ---

 -> Paso 1: Descargando datos de precios...
✅ Precios descargados (5020 filas).

 -> Paso 2: Descargando funding rates de Binance...
  -> Descargando funding rates para AVAX-USD (AVAXUSDT)...


/var/folders/z_/rcq9bsy13fzbphk_82m0p03r0000gn/T/ipykernel_51072/1487823124.py:41: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_raw = yf.download(new_ticker_list, start=start_date, end=end_date, progress=False)
/var/folders/z_/rcq9bsy13fzbphk_82m0p03r0000gn/T/ipykernel_51072/1487823124.py:46: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_processed = df_raw.stack(level=1).reset_index()


    ✅ 1507 registros de funding rate obtenidos
  -> Descargando funding rates para ATOM-USD (ATOMUSDT)...
    ✅ 1507 registros de funding rate obtenidos
  -> Descargando funding rates para NEAR-USD (NEARUSDT)...
    ✅ 1507 registros de funding rate obtenidos
  -> Descargando funding rates para FTM-USD (FTMUSDT)...
    ✅ 1507 registros de funding rate obtenidos
  -> Descargando funding rates para UNI-USD (UNIUSDT)...
    ✅ 1507 registros de funding rate obtenidos
  -> Descargando funding rates para AAVE-USD (AAVEUSDT)...
    ✅ 1507 registros de funding rate obtenidos
  -> Descargando funding rates para ALGO-USD (ALGOUSDT)...
    ✅ 1507 registros de funding rate obtenidos
  -> Descargando funding rates para TRX-USD (TRXUSDT)...
    ✅ 1507 registros de funding rate obtenidos
  -> Descargando funding rates para XLM-USD (XLMUSDT)...
    ✅ 1507 registros de funding rate obtenidos
  -> Descargando funding rates para XTZ-USD (XTZUSDT)...
    ✅ 1507 registros de funding rate obtenidos
✅ Funding

/var/folders/z_/rcq9bsy13fzbphk_82m0p03r0000gn/T/ipykernel_51072/1487823124.py:205: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  master_df = master_df.groupby('ticker', group_keys=False).apply(calculate_indicators)
/var/folders/z_/rcq9bsy13fzbphk_82m0p03r0000gn/T/ipykernel_51072/1487823124.py:268: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  master_df['funding_rate'] = master_df.groupby('ticker')['funding_rate'].fillna(method='ffill').fillna(0)
/var/folders/z_/rcq9bsy13fzbphk_82m0p03r0000gn/T/ipykernel_51072/14878231

In [11]:
# ====================================================================
# CELDA 2: VERIFICACIÓN DEL NUEVO DATAFRAME COMPLETO
# ====================================================================
import pandas as pd

print("--- [INICIO] Verificando 'master_df_new_universe_complete.parquet' ---")

try:
    # 1. Cargar el DataFrame que has creado
    data_path = 'dataframes/master_df_new_universe_complete.parquet'
    df_new_universe = pd.read_parquet(data_path)
    print(f"✅ DataFrame cargado con éxito desde '{data_path}'")

    # 2. Auditoría Rápida de los Datos
    print("\n" + "="*25 + " AUDITORÍA RÁPIDA " + "="*25)
    print(f" -> Shape del DataFrame: {df_new_universe.shape}")
    
    tickers_encontrados = sorted(df_new_universe['ticker'].unique())
    print(f"\n -> Tickers encontrados ({len(tickers_encontrados)}): {tickers_encontrados}")
    
    print("\n -> Columnas presentes (primeras 15):")
    print(df_new_universe.columns[:15].tolist())
    
    print("\n -> Conteo de valores nulos por columna:")
    print(df_new_universe.isnull().sum().to_string())
    
    print("\n--- [FIN] Verificación completada. ---")

except FileNotFoundError:
    print(f"❌ ERROR: No se pudo encontrar el archivo en la ruta '{data_path}'.")
except Exception as e:
    print(f"❌ ERROR inesperado durante la carga o verificación: {e}")

--- [INICIO] Verificando 'master_df_new_universe_complete.parquet' ---
✅ DataFrame cargado con éxito desde 'dataframes/master_df_new_universe_complete.parquet'

========================= AUDITORÍA RÁPIDA =========================
 -> Shape del DataFrame: (2710, 17)

 -> Tickers encontrados (10): ['AAVE-USD', 'ALGO-USD', 'ATOM-USD', 'AVAX-USD', 'FTM-USD', 'NEAR-USD', 'TRX-USD', 'UNI-USD', 'XLM-USD', 'XTZ-USD']

 -> Columnas presentes (primeras 15):
['timestamp', 'ticker', 'close', 'macd_signal', 'macd_hist', 'funding_rate', 'spy_close', 'vix_close', 'tnx_close', 'dxy_close', 'gc_close', 'cl_close', 'log_return', 'volatility_7d', 'price_to_ema_ratio']

 -> Conteo de valores nulos por columna:
timestamp              0
ticker                 0
close                  0
macd_signal            0
macd_hist              0
funding_rate           0
spy_close              0
vix_close              0
tnx_close              0
dxy_close              0
gc_close               0
cl_close               0


In [12]:
# ====================================================================
# CELDA 3: EJECUCIÓN DEL BACKTEST DE GENERALIZACIÓN
# ====================================================================
import vectorbt as vbt
import joblib
import pandas as pd
import numpy as np

print("--- [INICIO] Ejecutando backtest de generalización en el nuevo universo ---")

# --- 1. Cargar el Modelo Maestro ---
try:
    model_package = joblib.load('models/ULTRA_MODEL_PACKAGE.joblib')
    primary_model = model_package['primary_model_pipeline']
    meta_model = model_package['meta_model']
    optimal_threshold = model_package['optimal_threshold']
    model_features_list = model_package['feature_list']
    print("✅ Modelo Maestro cargado con éxito.")
except Exception as e:
    raise RuntimeError(f"❌ ERROR: No se pudo cargar el archivo del modelo. Error: {e}")

# --- 2. Preparar los datos finales ---
# Usamos el df_new_universe verificado en la celda anterior
X_oos = df_new_universe.copy()
if 'timestamp' in X_oos.columns:
    X_oos.set_index('timestamp', inplace=True)

all_tickers_oos = X_oos['ticker'].unique()
all_stats_oos = []

# --- 3. Bucle de Predicción y Backtesting por Ticker ---
for ticker in all_tickers_oos:
    print(f"\n{'='*20} PROCESANDO TICKER: {ticker} {'='*20}")
    
    # Seleccionamos los datos del ticker y nos aseguramos de tener solo las 15 features
    original_model_features = [col.split('__')[1] for col in model_features_list]
    ticker_X_oos = X_oos[X_oos['ticker'] == ticker][original_model_features]
    
    if ticker_X_oos.empty:
        continue

    # --- LÓGICA DE PREDICCIÓN (SIN ENTRENAMIENTO) ---
    primary_test_proba = primary_model.predict_proba(ticker_X_oos)
    primary_test_preds = np.argmax(primary_test_proba, axis=1)
    
    X_meta_test = pd.DataFrame({'primary_model_prob': primary_test_proba.max(axis=1)})
    meta_test_probs = meta_model.predict_proba(X_meta_test)[:, 1]
    
    entries = (meta_test_probs >= optimal_threshold)
    buy_signals = pd.Series((primary_test_preds == 1) & entries, index=ticker_X_oos.index)
    sell_signals = pd.Series((primary_test_preds == 0) & entries, index=ticker_X_oos.index)
    
    if buy_signals.sum() == 0 and sell_signals.sum() == 0:
        print(f" -> No se generaron operaciones para {ticker}.")
        continue

    # --- BACKTEST ---
    # Necesitamos los datos OHLC del DataFrame original para el backtest
    price_data_for_pf = df_new_universe[df_new_universe['ticker'] == ticker].set_index('timestamp')
    
    wf_portfolio = vbt.Portfolio.from_signals(
        close=price_data_for_pf['close'], 
        entries=buy_signals,
        exits=sell_signals,
        fees=0.002, sl_stop=0.05, tp_stop=0.05, init_cash=100000, freq='D')
    
    ticker_stats = wf_portfolio.stats()
    ticker_stats.name = ticker
    all_stats_oos.append(ticker_stats)
    print(f" -> RESULTADOS PARA {ticker}: Total Return: {ticker_stats['Total Return [%]']:.2f}%, Win Rate: {ticker_stats['Win Rate [%]']:.2f}%, Trades: {ticker_stats['Total Trades']}")

# --- INFORME FINAL ---
print(f"\n{'='*25} INFORME DE GENERALIZACIÓN (OUT-OF-SAMPLE) {'='*25}")
if not all_stats_oos:
    print("No se generaron estadísticas.")
else:
    final_stats_df_oos = pd.DataFrame(all_stats_oos)
    print(final_stats_df_oos[['Total Return [%]', 'Max Drawdown [%]', 'Win Rate [%]', 'Total Trades', 'Sharpe Ratio', 'Sortino Ratio']])

--- [INICIO] Ejecutando backtest de generalización en el nuevo universo ---
✅ Modelo Maestro cargado con éxito.

==================== PROCESANDO TICKER: AAVE-USD ====================


/Users/User/Library/CloudStorage/GoogleDrive-ignaciovct99@gmail.com/Mi unidad/Documentos/PROYECTOS/CRYPTONITA/cryptonita/.venv/lib/python3.11/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/User/Library/CloudStorage/GoogleDrive-ignaciovct99@gmail.com/Mi unidad/Documentos/PROYECTOS/CRYPTONITA/cryptonita/.venv/lib/python3.11/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/User/Library/CloudStorage/GoogleDrive-ignaciovct99@gmail.com/Mi unidad/Documentos/PROYECTOS/CRYPTONITA/cryptonita/.venv/lib/python3.11/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/User/Library/CloudStorage/GoogleDrive-ignaciovct99@gmail.com/Mi unidad/Documentos/PROYECTOS/CRYPTONITA/cryptonita/.venv/lib/python3.11/site-pa

 -> RESULTADOS PARA AAVE-USD: Total Return: 409.06%, Win Rate: 72.00%, Trades: 25

==================== PROCESANDO TICKER: ALGO-USD ====================
 -> RESULTADOS PARA ALGO-USD: Total Return: 298.41%, Win Rate: 79.17%, Trades: 25

==================== PROCESANDO TICKER: ATOM-USD ====================
 -> RESULTADOS PARA ATOM-USD: Total Return: 477.59%, Win Rate: 80.77%, Trades: 27

==================== PROCESANDO TICKER: AVAX-USD ====================
 -> RESULTADOS PARA AVAX-USD: Total Return: 101.80%, Win Rate: 73.91%, Trades: 23

==================== PROCESANDO TICKER: FTM-USD ====================
 -> RESULTADOS PARA FTM-USD: Total Return: 163.27%, Win Rate: 66.67%, Trades: 25

==================== PROCESANDO TICKER: NEAR-USD ====================
 -> RESULTADOS PARA NEAR-USD: Total Return: 341.55%, Win Rate: 79.17%, Trades: 24

==================== PROCESANDO TICKER: TRX-USD ====================
 -> RESULTADOS PARA TRX-USD: Total Return: 199.67%, Win Rate: 82.61%, Trades: 23

===

/Users/User/Library/CloudStorage/GoogleDrive-ignaciovct99@gmail.com/Mi unidad/Documentos/PROYECTOS/CRYPTONITA/cryptonita/.venv/lib/python3.11/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/User/Library/CloudStorage/GoogleDrive-ignaciovct99@gmail.com/Mi unidad/Documentos/PROYECTOS/CRYPTONITA/cryptonita/.venv/lib/python3.11/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/User/Library/CloudStorage/GoogleDrive-ignaciovct99@gmail.com/Mi unidad/Documentos/PROYECTOS/CRYPTONITA/cryptonita/.venv/lib/python3.11/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/User/Library/CloudStorage/GoogleDrive-ignaciovct99@gmail.com/Mi unidad/Documentos/PROYECTOS/CRYPTONITA/cryptonita/.venv/lib/python3.11/site-pa